In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tkinter as tk
from tkinter import messagebox
import folium
import webbrowser
from matplotlib.colors import LinearSegmentedColormap, rgb2hex
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib


In [4]:
data = pd.read_csv('../data/new_arabic_dialect_dataset.csv')

In [5]:
import pandas as pd

# Create an empty DataFrame to store the sampled sentences
new_data = pd.DataFrame(columns=['sentence', 'dialect', 'sentence_clean'])

# Iterate over each dialect
dialects = data['dialect'].unique()
for dialect in dialects:
    # Select 100 sentences from each dialect
    dialect_sentences = data[data['dialect'] == dialect].sample(n=1000, random_state=42)
    
    # Append the sampled sentences to the new DataFrame
    new_data = new_data.append(dialect_sentences)

# Save the new DataFrame to a CSV file
new_data = new_data.fillna('')
new_data.to_csv('../data/data_sample.csv', index=False)

C:\Users\Fares\AppData\Local\Temp\ipykernel_2376\15832291.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(dialect_sentences)
C:\Users\Fares\AppData\Local\Temp\ipykernel_2376\15832291.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(dialect_sentences)
C:\Users\Fares\AppData\Local\Temp\ipykernel_2376\15832291.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(dialect_sentences)
C:\Users\Fares\AppData\Local\Temp\ipykernel_2376\15832291.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(dialect_sentences)
C:\Users\Fares\A

In [6]:
from sklearn.model_selection import train_test_split

new_data = pd.read_csv('../data/data_sample.csv')

X = new_data.sentence_clean
y = new_data.dialect
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=new_data['dialect'])
print(X_train)

4753                                هـ بناخدوله وحدة الزوي
3194       صارت الساعه بدونك واقفه الدقايق حالفه تمر بلياك
17668    نادرا هتلاقي حد مجربش الحشيش أو الكحول ظنا منه...
16608                      بأشر وبعطيك ولا يهمك راسي حارتك
16217       فيه تلاعب المقطع ليش يقطع شوي أثناء رفع الكورة
                               ...                        
7471     نريد بعد واحد نزيه يجيب تزين واحد شغال يجيب شغ...
2980            مادري واله انتي راد تويت مادري عاد شنو كان
12933                             وين بيذيعون الإطلاق ومتى
16598                         قبل نمشي مكن تعطيني كم دقيقة
7632                        تستاهل زيون كم ريتويت حساب ميز
Name: sentence_clean, Length: 15200, dtype: object


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initizalize the vectorizer with max nr words and ngrams (1: single words, 2: two words in a row)
vectorizer_tfidf = TfidfVectorizer(ngram_range=(1,3))
# Fit the vectorizer to the training data
vectorizer_tfidf.fit(X_train)

TfidfVectorizer(ngram_range=(1, 3))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.model_selection import StratifiedKFold

vectorizer_tfidf = TfidfVectorizer()
classifier_tfidf_SVC = SVC(probability = True)

model_tfidf_SVC = Pipeline([
    ("vectorizer", vectorizer_tfidf),
    #("scaler", StandardScaler(with_mean=False)),
    ("classifier", classifier_tfidf_SVC)
])

param_grid = {
    "vectorizer__ngram_range": [(1, 1), (1, 2), (1, 3)],
    "classifier__C": [0.1, 1, 10]
}

# Perform grid search with cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(model_tfidf_SVC, param_grid, cv=cv, scoring="accuracy")
grid_search.fit(X_train, y_train)

In [ ]:
# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(best_params)
print(best_score)

In [ ]:
# Fit the model with the best parameters
model_tfidf_SVC.set_params(**best_params)
model_tfidf_SVC.fit(X_train, y_train)

# Save the model
joblib.dump(model_tfidf_SVC, '../models/model_tfidf_SVC.pkl')

In [7]:
from sklearn.metrics import accuracy_score, classification_report

saved_model_tfidf_SVC = joblib.load('../models/model_tfidf_SVC.pkl')

predicted_train_tfidf_SVC = saved_model_tfidf_SVC.predict(X_train)
accuracy_train_tfidf_SVC = accuracy_score(y_train, predicted_train_tfidf_SVC)
print('Accuracy Training data: {:.1%}'.format(accuracy_train_tfidf_SVC))

predicted_test_tfidf_SVC = saved_model_tfidf_SVC.predict(X_test)
accuracy_test_tfidf_SVC = accuracy_score(y_test, predicted_test_tfidf_SVC)
print('Accuracy Test data: {:.1%}'.format(accuracy_test_tfidf_SVC))

Accuracy Training data: 41.3%
Accuracy Test data: 41.8%


In [8]:
performance_metrics = classification_report(y_test, predicted_test_tfidf_SVC)
print(performance_metrics)

              precision    recall  f1-score   support

          AE       0.21      0.23      0.22       200
          BH       0.23      0.24      0.24       200
          DZ       0.45      0.45      0.45       200
          EG       0.56      0.56      0.56       200
          IQ       0.50      0.51      0.50       200
          JO       0.31      0.30      0.31       200
          KW       0.29      0.28      0.29       200
          LB       0.45      0.49      0.47       200
          LY       0.46      0.45      0.45       200
          MA       0.75      0.63      0.69       200
         MSA       0.88      0.84      0.86       200
          OM       0.26      0.30      0.28       200
          PL       0.33      0.35      0.34       200
          QA       0.29      0.30      0.29       200
          SA       0.30      0.34      0.31       200
          SD       0.64      0.54      0.59       200
          SY       0.38      0.33      0.35       200
          TN       0.70    

In [ ]:
model_data = pd.read_csv('../data/test_models.csv')
model_data = model_data.append({'Model': 'SVC', 'Performance Metrics': performance_metrics, 'Accuracy': accuracy_test_tfidf_SVC}, ignore_index=True)
model_data.to_csv('../data/test_models.csv', index=False)


In [ ]:
from sklearn.metrics import confusion_matrix

data = confusion_matrix(y_test, predicted_test_tfidf_SVC,labels=["MA", "DZ","TN","LY","EG","SD","LB","PL","SY","JO","OM","BH","KW","SA","AE","QA","IQ","YE","MSA"])
data_cm = pd.DataFrame(data, columns=["MA", "DZ","TN","LY","EG","SD","LB","PL","SY","JO","OM","BH","KW","SA","AE","QA","IQ","YE","MSA"], index = ["MA", "DZ","TN","LY","EG","SD","LB","PL","SY","JO","OM","BH","KW","SA","AE","QA","IQ","YE","MSA"])
data_cm.index.name = 'Actual'
data_cm.columns.name = 'Predicted'


f, ax = plt.subplots(figsize=(12, 12))
cmap = sns.cubehelix_palette(light=1, as_cmap=True)

sns.heatmap(data_cm, cbar=False, annot=True, cmap=cmap, square=True, fmt='.0f',
            annot_kws={'size':10})
plt.title('Actuals vs Predicted')
plt.savefig('../data/figs/SVC_matrix.png')
plt.show()

In [ ]:
def get_country_coordinates(country,proximity_score):
    # Define a dictionary with the latitude and longitude coordinates for each Arabic country
    country_coordinates = {
        "MA": [31.7917, -7.0926],
        "DZ": [36.7538, 3.0588],
        "TN": [33.8869, 9.5375],
        "LY": [26.3351, 17.2283],
        "EG": [26.8206, 30.8025],
        "SD": [15.5007, 32.5599],
        "LB": [33.8547, 35.8623],
        "PL": [33.8547, 35.8623],
        "SY": [34.8021, 38.9968],
        "JO": [30.5852, 36.2384],
        "OM": [21.4735, 55.9754],
        "BH": [26.0667, 50.5577],
        "KW": [29.3117, 47.4818],
        "SA": [23.8859, 45.0792],
        "AE": [23.4241, 53.8478],
        "QA": [25.3548, 51.1839],
        "IQ": [33.2232, 43.6793],
        "YE": [15.5527, 48.5164],
        "MSA": [20.0000, 40.0000]  # Placeholder coordinates for MSA
    }
    
    # Define the gradient colors from yellow to red
    colors = ["red","orange","yellow","green"]
    cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)

    # Map the proximity score to the gradient colormap
    color_rgb = cmap(proximity_score)

    # Convert the RGB color to hexadecimal format
    color_hex = rgb2hex(color_rgb)
    print(color_hex)

    return country_coordinates.get(country, [0, 0]), color_hex

In [ ]:
def predict_dialect():
    sentence = entry.get()
    minimum_probability = float('inf')
    maximum_probability = float('-inf')

    if sentence.strip() == '':
        messagebox.showerror("Error", "Please enter an Arabic sentence.")
        return

    prediction = saved_model_tfidf_SVC.predict([sentence])
    probabilities = saved_model_tfidf_SVC.predict_proba([sentence])[0]

    result = f"The predicted dialect is: {prediction}\n"
    result += "Prediction probabilities for each dialect:\n"
    for dialect, probability in zip(saved_model_tfidf_SVC.classes_, probabilities):
        result += f"{dialect}: {probability:.4f}\n"
        
        if probability < minimum_probability:
            minimum_probability = probability
        
        if probability > maximum_probability:
            maximum_probability = probability
        
        # Create a map centered around Arabic countries
    m = folium.Map(location=[24, 43], zoom_start=4)

    # Add a marker for the predicted dialect country
    predicted_country = prediction[0]
    folium.Marker(location=get_country_coordinates(predicted_country, 0)[0],
                  popup=f"Predicted: {predicted_country}",
                  icon=folium.Icon(color='green', icon='info-sign')).add_to(m)

    # Add heatmap layers for each Arabic country
    for dialect, probability in zip(saved_model_tfidf_SVC.classes_, probabilities):
        proximity_score = 1 - probability  # Calculate the proximity score
        normalized_score = (proximity_score - minimum_probability) / (maximum_probability - minimum_probability)
        coordinates, color = get_country_coordinates(dialect, proximity_score)  # Get coordinates and color
        folium.CircleMarker(location=coordinates,
                            radius=10,
                            color='{}'.format(color),
                            fill=True,
                            fill_color='{}'.format(color),
                            fill_opacity=0.7).add_to(m)

    # Save the map as an HTML file
    map_file = 'heatmap_SVC.html'
    m.save(map_file)

    # Open the HTML file in a web browser
    webbrowser.open(map_file)

    messagebox.showinfo("Prediction", result)

In [ ]:
root = tk.Tk()
root.title("Arabic Dialect Predictor - SVC Model")

label = tk.Label(root, text="Enter an Arabic sentence:")
label.pack()

entry = tk.Entry(root, width=50)
entry.pack()

button = tk.Button(root, text="Predict", command=predict_dialect)
button.pack()

root.mainloop()